   - treinar apenas como classificacao binaria no caso de psudo labels? nothing vs all
   - Aumentar o LR quando realizando o finetuning nos 10%?
   - cyclic lr no treinamento pseudolabels
   - usar a partica treino/este do chexnet


### Treinamento apenas com labels confiantes
    desconsiderar a perda em labels fora das faixas proximas de 0 e 1
    
- Escolhendo as melhores e piores pseudolabels para realizar o pre-treinamento
- Garantindo a mesma proporcao de labels presente no conjunto anotado
- A funcao de perda BCE desconsiderando as labels fora destas faixas
    - [1 0 1 0 -1 -1 -1] resulta em uma mascara que remove a perda nas labels -1
    
- Vemos que o desempenho da rede estudante nao ultrapassa o teacher
- O treinamento do finetuning com as imagens classificadas inicia melhor, como se os pseudo-labels funcionassem como um transfer-learning
- Porem o desempenho nao melhora apos a primeira epoch

- Experimentos
    - mantendo a mesma proporcao de labels como no conjunto original
    - diminuindo a proporcao e mantendo apenas labels com maior confianca
    
- O primeiro caso obteve um modelo baseado nas pseudolabels melhor, porem o finetuning ficou pior. Avaliar o porque




## TODO:
     - anotar o auroc durante o treinamento, mesmo para as pseudoplabels
     - realizar o finetuning com uma taxa de aprendizado mais baixa
     - testar mais data augmentation
     - utilizar os pseudolabels durante o treinamento [?] nao esta no artigo original mas existe em outros metodos
     
     


### TODOS
    
- treinamento com apenas um label
    - diminuir o dataset para uma classe
    - escolhendo uma label vs outros labels
    - label/outros ou labels/normal/nao-normal


# Notebook for making the intermediate predictions
 - using the 10% supervised models, we create the 

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

## Selecting the best 10% predictions for each label, for the unsupervised learning

In [8]:
with open('./runs/effusion_train05%_teacher_00/pseudo_labels95.txt', 'r') as file:
    data_preds = file.read()
    data_preds = data_preds.split('\n')

    images_names_pseudo = [entry.split(' ')[0] for entry in data_preds if len(entry) > 10]
    preds = [[float(p)  for p in entry.split(' ')[1:-1]]  for entry in data_preds if len(entry) > 10]
    preds = np.array(preds)


In [9]:
with open('./labels/binary_Effusion/train05%.txt', 'r') as file:
    data_labels = file.read()
    data_labels = data_labels.split('\n')
    images_names_finetune = [entry.split(' ')[0] for entry in data_labels]
    labels = [[int(p) for p in entry.split(' ')[1:]] for entry in data_labels if len(entry) > 10]
    labels = np.array(labels)

In [33]:
effusion_labels = labels[:,2]
effusion_ratio = np.sum(effusion_labels)/effusion_labels.shape[0]
print(f"The ratio betwen label 1 (effusion) and label 0 (normal) is {effusion_ratio}")

The ratio betwen label 1 (effusion) and label 0 (normal) is 0.15614103060829135


In [34]:
filtered_preds = -1*np.ones([preds.shape[0], 14]) # Start ignoring all entries

k_best = 0.5 #Use the k_best% best low and high predictioins
k_high_predictions = int(k_best*effusion_ratio*preds.shape[0])
k_low_predictions = int(k_best*(1-effusion_ratio)*preds.shape[0])

print(f"From a total of {preds.shape[0]} predicitons, we are taking : ")
print(f" -- Getting {k_high_predictions}   [{round(100*k_high_predictions/preds.shape[0],2)}% of total] high predictions")
print(f" -- Getting {k_low_predictions}  [{round(100*k_low_predictions/preds.shape[0],2)}% of total]low predictions")

From a total of 49085 predicitons, we are taking : 
 -- Getting 3832   [7.81% of total] high predictions
 -- Getting 20710  [42.19% of total]low predictions


In [35]:

n_preds = preds[:, 0]
n_argsorted = np.argsort(n_preds)
k_highprob = n_preds[n_argsorted[-k_high_predictions]] # the lower bound of probability to be included in the ssl training
k_lowprob = n_preds[n_argsorted[k_low_predictions]] # the lower bound of probability to be included in the ssl training
print(f"The probability limits are:")
print(f" -- {k_highprob} for label 1")
print(f" -- {k_lowprob} for label 0")


filtered_preds[preds[:,0] >= k_highprob, 2] = 1
filtered_preds[preds[:,0] <= k_lowprob, 2] = 0

The probability limits are:
 -- 0.8495872616767883 for label 1
 -- 0.005060830619186163 for label 0


In [37]:
data = ''
for image, filtered_pred in zip(images_names_pseudo, filtered_preds.astype(np.int8)):
    if filtered_pred[2]==-1:
        continue
    data += image + ' ' + ' '.join([str(p) for p in filtered_pred])
    data += '\n'
    
with open('./runs/effusion_train05%_teacher_00/pseudo_labels95_filtered.txt', 'w') as file:
    file.write(data)

# Options 3 - ignore uncertain predictions

In [251]:
from torch import nn
import torch

In [252]:
criterion = nn.functional.binary_cross_entropy_with_logits


In [278]:
target = torch.FloatTensor([[ 0.,  0., 1.,  1.,  -1.,  -1.],
                            [ 0.,  0., 1.,  1.,  -1.,  -1.]])

output = torch.FloatTensor([[0.4037, 0.5437, 0.5194, 0.7067, 0.3060, 0.4174],
                            [0.3490, 0.5077, 0.4751, 0.7386, 0.3352, 0.4569]])

loss_weight = torch.gt(target, -0.5)
loss = criterion(output, target, loss_weight)

print(loss_weight)
print(loss*6)

tensor([[ True,  True,  True,  True, False, False],
        [ True,  True,  True,  True, False, False]])
tensor(2.7601)


In [279]:
target = torch.FloatTensor([[ 0.,  0., 1.,  1.],
                            [ 0.,  0., 1.,  1.]])

output = torch.FloatTensor([[0.4037, 0.5437, 0.5194, 0.7067 ],
                            [0.3490, 0.5077, 0.4751, 0.7386]])

loss_weight = torch.gt(target, -0.5)
loss = criterion(output, target, loss_weight)

print(loss_weight)
print(loss*4)

tensor([[True, True, True, True],
        [True, True, True, True]])
tensor(2.7601)


# Option 1 - keep proportion of classes

In [37]:
N = preds.shape[0]
for L in range(14):
    pseudolabels = np.sort(preds[:, L], )
    min_prob = pseudolabels[N-int(N*label_ratios[L])-1]
    preds[:, L] = 1.0*(preds[:, L] >min_prob)

# Option 2 - keep those above mean

In [93]:
preds = 1.0*(preds >preds.mean(axis=0))

In [95]:
train_list90_pseudo = []
for pair in zip(images_names, preds):
    pred = f"{pair[0]} {' '.join([str(int(p)) for p in pair[1]])}"
    train_list90_pseudo.append(pred)
    

In [96]:
with open('./labels/train_list90_pseudo2.txt', 'w') as file:
    for pred in train_list90_pseudo:
        file.write(pred+'\n')

In [44]:
# encoding: utf-8

"""
The main CheXNet model implementation.
"""


import os
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from read_data import ChestXrayDataSet
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import progressbar
from torch.optim.lr_scheduler import ReduceLROnPlateau

from model import Resnet50, DenseNet121
import time
import json

CKPT_PATH = './snapshots/model.pth.tar'
N_CLASSES = 14
CLASS_NAMES = [ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
                'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']
DATA_DIR = '/media/roberto/external/Data/images/images'
TRAIN_IMAGE_LIST = './densenet_supervised10%_training/train_list90%_pseudolabels.txt'
VAL_IMAGE_LIST = './labels/val_list.txt'

BATCH_SIZE = 16
EPOCHS = 20
LR = 1e-4



normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])

train_dataset = ChestXrayDataSet(data_dir=DATA_DIR,
                                image_list_file=TRAIN_IMAGE_LIST,
                                transform=transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomRotation(15),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])
                                ]))
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=8, pin_memory=True)

normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])

val_dataset = ChestXrayDataSet(data_dir=DATA_DIR,
                                image_list_file=VAL_IMAGE_LIST,
                               transform=transforms.Compose([
                                   transforms.Resize(256),
                                   transforms.TenCrop(224),
                                   transforms.Lambda
                                   (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                   transforms.Lambda
                                   (lambda crops: torch.stack([normalize(crop) for crop in crops]))
                               ]))


val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=8, pin_memory=True)



ValueError: invalid literal for int() with base 10: '0.07252079248428345'

In [34]:
for img, target in val_loader:
    a = img
    b = target
    break

In [42]:
' '.join([str(p) for p in b[0].tolist()])

'0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0'

In [33]:
"/media/roberto/external/Data/images/images/00021321_013.png".split("/")[-1]

'00021321_013.png'